In [1]:
import newspaper
import pandas as pd

In [2]:
link = 'https://www.cnn.com'
# Scans the webpage and finds all the links on it.
page_features = newspaper.build(link, language='en', memoize_articles=False)
# Initialize a list for article titles and text.


In [5]:
from tqdm.auto import tqdm

def extract(num_articles: int = 25) -> pd.DataFrame:
    link = 'https://www.cnn.com'
    # Scans the webpage and finds all the links on it.
    page_features = newspaper.build(
        link, 
        language='en', 
        memoize_articles=False
    )

    articles = tqdm(page_features.articles[:num_articles])
    data = []
    for article in articles:
        try:
            # Each article must be downloaded, then parsed individually.
            # This loads the text and title from the webpage to the object.
            article.download()
            article.parse()

            if not article.url.startswith('https://edition.cnn.com'):
                # Keep the text, title and URL from the article and append to a list.
                data.append({
                    'title':article.title,
                    'article':article.text,
                    'url': article.url})
                print(f"Successful get {article.title}")
        except Exception as e:
            # If, for any reason the download fails, continue the loop.
            print("Article Download Failed: " + str(e))

    df = pd.DataFrame.from_dict(data)
    print(df)
    df = df[df['article'].str.len() < 10000]
    return df

In [6]:
df = extract(25)
df

  0%|          | 0/25 [00:00<?, ?it/s]

Successful get CNN Business
Successful get Live updates: Russia's war in Ukraine, deadly Lviv missile strike
Successful get Titan submersible implosion: OceanGate suspends operations after deadly disaster
Successful get First Alzheimer’s drug to slow disease progression expected to get full FDA approval
Successful get Threads: Everything to know about Instagram's app to rival Twitter
Successful get Keke Palmer’s partner slammed for shaming her over outfit
Successful get Rudy Farias: A Texas man reported missing as a teen 8 years ago has been at home with his mom all along, police say
Successful get A revered and powerful female leader revealed by new method to determine sex of old bones
Successful get Ancient shrimplike predator was misunderstood, according to new research
Successful get Cut marks on fossilized bone may reveal ancient cannibalism
Successful get Dow tumbles on red-hot US job market report
Successful get US job openings dropped below 10 million in May
Successful get Why 

,title,article,url
0,CNN Business,1. How relevant is this ad to you?\n\nVideo pl...,https://www.cnn.com/business/media
1,"Live updates: Russia's war in Ukraine, deadly ...",The United States is expected to announce a ne...,https://www.cnn.com/europe/live-news/russia-uk...
2,Titan submersible implosion: OceanGate suspend...,1. How relevant is this ad to you?\n\nVideo pl...,https://www.cnn.com/2023/07/06/us/oceangate-su...
3,First Alzheimer’s drug to slow disease progres...,CNN —\n\nThe US Food and Drug Administration i...,https://www.cnn.com/2023/07/06/health/leqembi-...
4,Threads: Everything to know about Instagram's ...,New York CNN —\n\nFacebook-parent Meta on Wedn...,https://www.cnn.com/2023/07/06/tech/instagram-...
5,Keke Palmer’s partner slammed for shaming her ...,CNN —\n\nThe father of Keke Palmer’s son is ge...,https://www.cnn.com/2023/07/06/entertainment/k...
6,Rudy Farias: A Texas man reported missing as a...,CNN —\n\nA Texas man reported missing eight ye...,https://www.cnn.com/2023/07/06/us/rudy-farias-...
7,A revered and powerful female leader revealed ...,Editor’s Note: Sign up for CNN’s Wonder Theory...,https://www.cnn.com/2023/07/06/europe/powerful...
8,"Ancient shrimplike predator was misunderstood,...",Editor’s Note: Sign up for CNN’s Wonder Theory...,https://www.cnn.com/2023/07/04/world/cambrian-...
9,Cut marks on fossilized bone may reveal ancien...,Editor’s Note: Sign up for CNN’s Wonder Theory...,https://www.cnn.com/2023/06/26/world/fossilize...


In [5]:
import requests

def post_req(url:str, params = None, data: dict = None) -> requests.Response:
    headers = {
        "Content-type": "application/json",
    }
    try:
        r = requests.post(url, params = params, json=data, headers=headers)
    except Exception as e:
        print("error happen here:\n", e)
    else:
        if r.status_code == 200:
            return r
        else:
            print("request code is not 200")

In [6]:
def transform(model_uri: str, df: pd.DataFrame) -> pd.DataFrame:
    df = df.sample(10)  # TODO: delete this
    # generate text summarization
    r = post_req(
        url = f"http://{model_uri}:8000/article/summarize_batch", 
        data = dict(
            articles=dict(articles=df['article'].tolist()),
            config=dict(num_beans=8, temperature=1.0)    
        )
    )
    df.loc[:, 'summary'] = r.json() 
    return df

In [7]:
df = transform("172.17.0.1", df)
df

,title,article,url,summary
79,"Gannett sues Google, alleging it has an online...","CNN —\n\nGannett, the largest newspaper publis...",https://www.cnn.com/2023/06/20/tech/gannett-ne...,The publisher of USA Today and more than 200 l...
32,JPMorgan fined $4 million by SEC for deleting ...,New York CNN —\n\nThe Securities and Exchange ...,https://www.cnn.com/2023/06/23/business/jpmorg...,JPMorgan Chase fined JPMorgan Chase $4 million...
60,Abortion is ancient history and that matters t...,"CNN —\n\nAbortion today, at least in the Unite...",https://www.cnn.com/2023/06/23/health/abortion...,"Egyptian papyrus, Roman coins, medieval biogra..."
84,Victor Wembanyama Celebrated by Fans After Goi...,TIMOTHY A. CLARY/AFP via Getty Images\n\nAfter...,https://bleacherreport.com/articles/10080347-v...,Wembanyama is expected to be the No.
68,Fact check: Big differences between Hunter Bid...,Washington CNN —\n\nMany Republicans have crit...,https://www.cnn.com/2023/06/23/politics/fact-c...,Some Republicans have focused on how the prose...
54,The world’s 50 best restaurants revealed,CNN —\n\nPeru’s capital city – and undoubtedly...,https://www.cnn.com/travel/worlds-50-best-rest...,The 50 Best Restaurants awards took the No.
71,A timeline of the abortion access landscape on...,How the frenzy of legal actions shifted the la...,https://www.cnn.com/interactive/2023/06/us/abo...,The Supreme Court has issued its ruling in Dob...
4,The ultra-wealthy have dangerous pastimes. Who...,"New York CNN —\n\nThroughout history, humans h...",https://www.cnn.com/2023/06/24/business/extrem...,This week's catastrophic implosion of the Ocea...
63,Why Chinese students are taking graduation pho...,Hong Kong CNN —\n\nOne photo shows the young w...,https://www.cnn.com/2023/06/23/china/china-gra...,Chinese social media has become awash with ton...
9,Analysis: Moscow is at risk of losing its iron...,CNN —\n\nThis just does not happen in Vladimir...,https://www.cnn.com/2023/06/24/europe/russia-p...,Russian president is facing the most serious t...


In [10]:
import mysql.connector

with mysql.connector.connect(
    host='localhost',
    database='summarizer',
    user='root',
    password='luntaixia'
) as conn:
    with conn.cursor() as cursor:
        mysql_quey = """
        INSERT INTO summarizer.batch_summarization (title, article, `url`, summary) 
        VALUES (%s, %s, %s, %s)
        """
        params = list(df.itertuples(index=False, name=None))
        cursor.executemany(mysql_quey, params)
        conn.commit()

In [9]:
from sqlalchemy import create_engine

def load(df: pd.DataFrame, conn_str:str):
    engine = create_engine(conn_str)

    with engine.begin() as conn:
        df.to_sql(
            name = 'batch_summarization', 
            con = conn, 
            schema = 'summarizer',
            if_exists = 'append',
            index = False,
            method = 'multi',
            chunksize = 1000
        )
